In [1]:
import pdfplumber
import sqlite3
import pandas as pd

In [17]:
# PDF file path
pdf_file = "NEXT_SENIORITY_LIST.pdf"

# Output SQLite DB file
db_file = "database.db"

# Open PDF and extract tables
all_rows = []
with pdfplumber.open(pdf_file) as pdf:

    for page in pdf.pages:
        table = page.extract_table()
        if table:
            df = pd.DataFrame(table[0:])
            all_rows.append(df)

     

In [21]:
print(len(all_rows), "pages processed.")
all_rows[4].info()


717 pages processed.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       26 non-null     object
 1   1       26 non-null     object
 2   2       26 non-null     object
 3   3       26 non-null     object
 4   4       26 non-null     object
 5   5       26 non-null     object
 6   6       26 non-null     object
 7   7       26 non-null     object
dtypes: object(8)
memory usage: 1.8+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   27                    25 non-null     object
 1   SHASHIKANTA           25 non-null     object
 2   RJJP199718016057      25 non-null     object
 3   03-10-1970            25 non-null     object
 4   OBC                   25 non-null     object
 5   English (Compulsary)  25 non-null     object
 6   DPC                   25 non-null     object
 7   2016-17               25 non-null     object
dtypes: object(8)
memory usage: 1.7+ KB


In [22]:
full_df = pd.concat(all_rows, ignore_index=True)

In [24]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18607 entries, 0 to 18606
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       18607 non-null  object
 1   1       18607 non-null  object
 2   2       18607 non-null  object
 3   3       18607 non-null  object
 4   4       18607 non-null  object
 5   5       18607 non-null  object
 6   6       18607 non-null  object
 7   7       18607 non-null  object
dtypes: object(8)
memory usage: 1.1+ MB


In [ ]:
full_df.drop(index=0, inplace=True)  # Drop row at index 0



,0,1,2,3,4,5,6,7
1,1,UNKAR LAL MEENA,RJKA199626021433,24-06-1970,ST,Hindi Literature,DPC,2016-17
2,2,PRATIBHA PATHAK,RJBI199809000454,09-09-1973,GEN,Hindi Literature,DPC,2016-17
3,3,BAHADUR SINGH,RJBP198607000115,02-12-1965,SC,Hindi Literature,DPC,2016-17
4,4,BABITA YADAV,RJJJ199823008199,09-03-1977,OBC,History,DPC,2016-17
5,5,GANESH KUMAR JOSHI,RJJP199117022975,25-09-1971,GEN,English (Compulsary),DPC,2016-17


In [30]:
full_df.columns = ['SrNo', 'Name', 'ID', 'DOB', 'Category', 'Subject', 'Center', 'Year']

# Filter
filtered_df = full_df[full_df['Category'].str.endswith('GEN')]

# Count how many matched
count = len(filtered_df)

print(f"Number of candidates: {count}")
filtered_df.head(10)

Number of candidates: 4649


,SrNo,Name,ID,DOB,Category,Subject,Center,Year
2,2,PRATIBHA PATHAK,RJBI199809000454,09-09-1973,GEN,Hindi Literature,DPC,2016-17
5,5,GANESH KUMAR JOSHI,RJJP199117022975,25-09-1971,GEN,English (Compulsary),DPC,2016-17
6,6,MEENAKSHI\nSHROTRIYA,RJJP199418014572,04-04-1969,GEN,Hindi Literature,DPC,2016-17
13,13,ritu gupta,rjpg199630026249,02-07-1971,GEN,Physics (Science Stream),DPC,2016-17
15,15,OM PRAKASH SHARMA,RJBW199108009326,19-05-1967,GEN,English (Compulsary),DPC,2016-17
18,18,SUDHA ASTHANA,RJJP199717017147,22-07-1971,GEN,Hindi (Compulsary),DPC,2016-17
22,22,Jyoti Pareek,RJUD199837018258,29-11-1973,GEN,English (Compulsary),DPC,2016-17
26,26,RAJ RAJESHWARI\nRATHORE,RJUD199437006711,23-07-1969,GEN,History,DPC,2016-17
28,28,REKHA YADAV,RJBW199708002666,05-09-1972,GEN,History,DPC,2016-17
29,29,manisha mehta,rjjp199117012808,12-04-1969,GEN,English (Compulsary),DPC,2016-17


In [32]:
category_counts = full_df['Category'].value_counts()
category_counts

Category
OBC    8036
GEN    4649
SC     2990
ST     2623
SBC     305
          3
Name: count, dtype: int64

In [33]:
import sqlite3
import pandas as pd

# Step 1: Connect to SQLite database (creates file if it doesn't exist)
conn = sqlite3.connect('candidates.db')
cursor = conn.cursor()

# Step 3: Create a table dynamically based on columns
columns_sql = ", ".join([f'"{col}" TEXT' for col in full_df.columns])  # All columns as TEXT
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute(f"CREATE TABLE candidates ({columns_sql})")

# Step 4: Insert data row by row
for _, row in full_df.iterrows():
    placeholders = ','.join(['?'] * len(row))
    cursor.execute(f"INSERT INTO candidates VALUES ({placeholders})", tuple(row))

# Step 5: Commit & close connection
conn.commit()
conn.close()

print("✅ DataFrame successfully written to 'candidates' table in candidates.db")

✅ DataFrame successfully written to 'candidates' table in candidates.db


In [34]:
conn = sqlite3.connect('candidates.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM candidates LIMIT 5;")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

('1', 'UNKAR LAL MEENA', 'RJKA199626021433', '24-06-1970', 'ST', 'Hindi Literature', 'DPC', '2016-17')
('2', 'PRATIBHA PATHAK', 'RJBI199809000454', '09-09-1973', 'GEN', 'Hindi Literature', 'DPC', '2016-17')
('3', 'BAHADUR SINGH', 'RJBP198607000115', '02-12-1965', 'SC', 'Hindi Literature', 'DPC', '2016-17')
('4', 'BABITA YADAV', 'RJJJ199823008199', '09-03-1977', 'OBC', 'History', 'DPC', '2016-17')
('5', 'GANESH KUMAR JOSHI', 'RJJP199117022975', '25-09-1971', 'GEN', 'English (Compulsary)', 'DPC', '2016-17')
